In [ ]:
import numpy as np
import tensorflow as tf
import gzip
import pickle
import matplotlib.pyplot as plt

In [ ]:
# python 3.6
with gzip.open('datasets/mnist.pkl.gz', 'rb') as fp:
    train_set, valid_set, test_set = pickle.load(fp, encoding='latin1')

In [ ]:
train_X, train_y = train_set
valid_X, valid_y = valid_set

In [ ]:
element_id = 50
plt.imshow(train_X[element_id].reshape(28, 28), cmap='Greys')
plt.title("This is `{}`".format(train_y[element_id]))
plt.show()

In [ ]:
# Functional API: https://www.tensorflow.org/api_docs/python/tf/layers/dense
# Class API: https://www.tensorflow.org/api_docs/python/tf/layers/Dense
def neural_net(input_layer, hidden_layers=[1024, 256, 64], num_classes=10, 
               with_dropout=True,
               with_batch_norm=False,
               activation=tf.nn.relu):
        
    layer = input_layer
    
    if with_batch_norm:
        layer = tf.layers.batch_normalization(layer)


    for l, units in enumerate(hidden_layers):
        layer = tf.layers.dense(layer, units=units, activation=activation)
        if with_dropout:
            layer = tf.layers.dropout(layer, rate=0.8)

    logits = tf.layers.dense(layer, units=num_classes)
    return logits

In [ ]:
with tf.variable_scope('input_placeholder'):
    X_placeholder = tf.placeholder(shape=(None, 784), dtype=tf.float32, name='inputX')
    Y_placeholder = tf.placeholder(shape=(None), dtype=tf.int64, name='inputY')

In [ ]:
BATCH_SIZE = 256
NUM_EPOCH = 25
learning_rate = 0.0004

batch_size = tf.placeholder(dtype=tf.int64)
dataset = tf.data.Dataset.from_tensor_slices((X_placeholder, Y_placeholder))
dataset = dataset.batch(batch_size)
data_iterator = dataset.make_initializable_iterator()
input_layer, labels = data_iterator.get_next()

logits = neural_net(input_layer)
loss_op = tf.losses.sparse_softmax_cross_entropy(labels, logits)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

pred_classes_op = tf.argmax(logits, axis=1)
# acc_op = tf.metrics.accuracy(labels, pred_classes_op)

correct_pred = tf.equal(pred_classes_op, labels)
acc_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

def report_train_valid_accuracy(epoch, sess):
    sess.run(data_iterator.initializer, feed_dict={X_placeholder: train_X, 
                                                   Y_placeholder: train_y,
                                                   batch_size: train_X.shape[0]})
    loss, acc = sess.run([loss_op, acc_op])
    print(f"epoch={epoch} loss={loss:0.5f} acc={acc:0.5f} ", end='')
    

        
    sess.run(data_iterator.initializer, feed_dict={X_placeholder: valid_X, 
                                                       Y_placeholder: valid_y,
                                                       batch_size: valid_X.shape[0]})
        
    loss, acc = sess.run([loss_op, acc_op]) 
    print(f"test_loss={loss:0.5f} test_acc={acc:0.5f}")
    

with tf.Session() as sess:
    # For tf.metrics.accuracy
    # sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    
    report_train_valid_accuracy(0, sess)
    
    
    for epoch in range(NUM_EPOCH):
        sess.run(data_iterator.initializer, feed_dict={X_placeholder: train_X, 
                                                       Y_placeholder: train_y,
                                                       batch_size: BATCH_SIZE})
        while True:
            try:
                loss, _= sess.run([loss_op, train_op])
            except tf.errors.OutOfRangeError:
                break
                        
        report_train_valid_accuracy(epoch+1, sess)